In [3]:
f = '/home/george-vengrovski/Documents/projects/tweety_bert_paper/gtruth_pst_data.txt'

def compute_markovian_transitions_with_percentages(file_path):
    from collections import defaultdict
    import operator

    # Initialize a dictionary to hold the count of each transition
    transitions = defaultdict(int)

    # Read the file
    with open(file_path, 'r') as file:
        for line in file:
            # Parse characters, strip to remove newline and split by comma
            characters = line.strip().split(',')

            # Compute transitions
            for i in range(len(characters) - 1):
                current_char = characters[i]
                next_char = characters[i + 1]

                # Concatenate the current and next characters to form the transition
                transition = f'{current_char}->{next_char}'

                # Increment the count of this transition
                transitions[transition] += 1

    # Calculate the total number of transitions
    total_transitions = sum(transitions.values())

    # Sort the transitions by count in descending order and get the top ten
    sorted_transitions = sorted(transitions.items(), key=operator.itemgetter(1), reverse=True)[:10]

    # Print the ten most common transitions with raw count and percentage
    for transition, count in sorted_transitions:
        percentage = (count / total_transitions) * 100  # Calculate percentage
        print(f'Transition: {transition}, Count: {count}, Percentage: {percentage:.2f}%')

# Replace 'your_file_path.txt' with the actual path to your text file


# Replace 'your_file_path.txt' with the actual path to your text file
compute_markovian_transitions_with_percentages(f)


Transition: C->F, Count: 264, Percentage: 10.57%
Transition: F->A, Count: 205, Percentage: 8.21%
Transition: A->C, Count: 169, Percentage: 6.77%
Transition: K->R, Count: 168, Percentage: 6.73%
Transition: F->C, Count: 122, Percentage: 4.89%
Transition: B->G, Count: 111, Percentage: 4.45%
Transition: A->B, Count: 106, Percentage: 4.25%
Transition: Q->K, Count: 98, Percentage: 3.92%
Transition: H->J, Count: 96, Percentage: 3.84%
Transition: R->I, Count: 72, Percentage: 2.88%


In [15]:
import numpy as np

# Load the .npy file
f = np.load("/home/george-vengrovski/Documents/projects/tweety_bert_paper/hdbscanlabels.npy")

# # Slice the first 1000 elements
# first_1000_elements = f

# # Save the sliced array to a text file named 'temp.txt'
# with open('temp.txt', 'w') as file:
#     for item in first_1000_elements:
#         file.write(f"{item},")

# # Confirmation message
# print("The first 1000 elements have been saved to 'temp.txt'.")


In [14]:
from collections import Counter

def majority_vote(data):
    # Function to find the majority element in a window
    def find_majority(window):
        count = Counter(window)
        majority = max(count.values())
        for num, freq in count.items():
            if freq == majority:
                return num
        return window[1]  # Return the middle element if no majority found

    # Ensure the input data is in list form
    if isinstance(data, str):
        data = [int(x) for x in data.split(',') if x.strip().isdigit()]

    # Initialize the output array with a padding at the beginning
    output = [data[0]]  # Pad with the first element

    # Apply the majority vote on each window
    for i in range(1, len(data) - 1):  # Start from 1 and end at len(data) - 1 to avoid index out of range
        window = data[i-1:i+2]  # Define the window with size 3
        output.append(find_majority(window))

    # Pad the output array at the end to match the input array size
    output.append(data[-1])

    return output


majority_vote_arr = majority_vote(f)

In [ ]:
def syllable_to_phrase_labels(arr, silence=-1):
    new_arr = np.array(arr, dtype=int)
    current_syllable = None
    start_of_phrase_index = None
    first_non_silence_label = None  # To track the first non-silence syllable

    for i, value in enumerate(new_arr):
        if value != silence and value != current_syllable:
            if start_of_phrase_index is not None:
                new_arr[start_of_phrase_index:i] = current_syllable
            current_syllable = value
            start_of_phrase_index = i
            
            if first_non_silence_label is None:  # Found the first non-silence label
                first_non_silence_label = value

    if start_of_phrase_index is not None:
        new_arr[start_of_phrase_index:] = current_syllable

    # Replace the initial silence with the first non-silence syllable label
    if new_arr[0] == silence and first_non_silence_label is not None:
        for i in range(len(new_arr)):
            if new_arr[i] != silence:
                break
            new_arr[i] = first_non_silence_label

    return new_arr

def reduce_phrases(arr, remove_silence=True):
    current_element = arr[0]
    reduced_list = [] 

    for i, value in enumerate(arr):
        if value != current_element:
            reduced_list.append(current_element)
            current_element = value 

        # append last phrase
        if i == len(arr) - 1:
            reduced_list.append(current_element)

    if remove_silence == True:
        reduced_list = [value for value in reduced_list if value != 0]

    return np.array(reduced_list)


majority_vote_arr